# Carga de Datamart

In [1]:
import os
import re
import pandas as pd
import pyodbc
from tqdm import tqdm

In [2]:
import pandas as pd
import pyodbc
import numpy as np

#----------------------------------- Configura la conexión a SQL Server DataMart --------------------------------
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'  # Asegúrate de tener el controlador adecuado
    'SERVER=localhost\SQLEXPRESS01;'                  # Reemplaza con el nombre de tu servidor
    'DATABASE=datamart_importaciones_2;'           # Reemplaza con el nombre de tu base de datos
    'Trusted_Connection=yes;'                           # Reemplaza con tu contraseña
)
cursor = conn.cursor()


In [3]:
df = pd.read_csv("final_finalisimo_ahora_si.csv")

C:\Users\max_1\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (3,4,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
df['FECH_INGSI'] = pd.to_datetime(df['FECH_INGSI'],format='%Y%m%d')
df['FECH_LLEGA'] = pd.to_datetime(df['FECH_LLEGA'],format='%Y%m%d')

ValueError: time data '0' does not match format '%Y%m%d' (match)

In [11]:
df.head()

,CODI_ADUAN,NUME_CORRE,FECH_INGSI,TIPO_DOCUM,LIBR_TRIBU,DNOMBRE,CODI_AGENT,FECH_LLEGA,VIA_TRANSP,EMPR_TRANS,...,Jurisdiccion_Almacen,Cpais,Dpais,Cpuerto,Dpuerto,Descripcion_Aduana,Clase_Bulto,Razon Social,Codigo,Estado_Mercancia
0,235,135859,2024-09-03,4,20606352825,2M DIESEL S.A.C.,14,20240825,4,UX,...,A NIVEL NACIONAL,TR,TURKEY,IST,ISTANBUL,AEREA Y POSTAL EX-IAAC,BULTOS ...,AIR EUROPA LINEAS AEREAS S.A. SUCURSAL DEL PERU,10,NUEVO/BUENO
1,235,135859,2024-09-03,4,20606352825,2M DIESEL S.A.C.,14,20240825,4,UX,...,A NIVEL NACIONAL,TR,TURKEY,IST,ISTANBUL,AEREA Y POSTAL EX-IAAC,BULTOS ...,AIR EUROPA LINEAS AEREAS S.A. SUCURSAL DEL PERU,10,NUEVO/BUENO
2,235,137787,2024-09-06,4,20503101913,A & N PROYECTOS S.A.C. - A & N S.A.C.,3728,20240907,4,AF,...,No mencionado,ES,SPAIN,BCN,BARCELONA,AEREA Y POSTAL EX-IAAC,BULTOS ...,SOCIETE AIR FRANCE SUCURSAL EN EL PERU,10,NUEVO/BUENO
3,235,137787,2024-09-06,4,20503101913,A & N PROYECTOS S.A.C. - A & N S.A.C.,3728,20240907,4,AF,...,No mencionado,ES,SPAIN,BCN,BARCELONA,AEREA Y POSTAL EX-IAAC,BULTOS ...,SOCIETE AIR FRANCE SUCURSAL EN EL PERU,10,NUEVO/BUENO
4,235,137787,2024-09-06,4,20503101913,A & N PROYECTOS S.A.C. - A & N S.A.C.,3728,20240907,4,AF,...,No mencionado,ES,SPAIN,BCN,BARCELONA,AEREA Y POSTAL EX-IAAC,BULTOS ...,SOCIETE AIR FRANCE SUCURSAL EN EL PERU,10,NUEVO/BUENO


In [21]:
df.columns

Index(['CODI_ADUAN', 'NUME_CORRE', 'FECH_INGSI', 'TIPO_DOCUM', 'LIBR_TRIBU',
       'DNOMBRE', 'CODI_AGENT', 'FECH_LLEGA', 'VIA_TRANSP', 'EMPR_TRANS',
       'CODI_ALMA', 'FECH_RECEP', 'PAIS_ORIGE', 'PAIS_ADQUI', 'PUER_EMBAR',
       'FECH_EMBAR', 'NUME_SERIE', 'DESC_COMER', 'FLE_DOLAR', 'SEG_DOLAR',
       'PESO_NETO', 'PESO_BRUTO', 'UNID_FIQTY', 'UNID_FIDES', 'QUNICOM',
       'TUNICOM', 'SEST_MERCA', 'CANT_BULTO', 'CLASE', 'Razon_Social_Almacen',
       'Jurisdiccion_Almacen', 'Cpais', 'Dpais', 'Cpuerto', 'Dpuerto',
       'Descripcion_Aduana', 'Clase_Bulto', 'Razon Social', 'Codigo',
       'Estado_Mercancia', 'TRANSPORTE'],
      dtype='object')

In [50]:
max_longitud = df['DESC_COMER'].apply(lambda x: len(str(x))).max()
print(f"La máxima longitud en caracteres es: {max_longitud}")

La máxima longitud en caracteres es: 90


In [60]:
df['CANT_BULTO'] = df['CANT_BULTO'].round(2) 
df['PESO_NETO'] = df['PESO_NETO'].round(2) 
df['PESO_BRUTO'] = df['PESO_BRUTO'].round(2) 

In [82]:
df.to_csv("final_finalisimo_ahora_si.csv",index=False)

## Creacion de tablas

In [5]:
cursor.execute("""
IF OBJECT_ID('Dim_FechaPuerto', 'U') IS NULL
    CREATE TABLE Dim_FechaPuerto (
        id_FechaPuerto INT PRIMARY KEY,
        Dia INT NULL,
        Mes INT NULL,
        Año INT NULL,
        Trimestre INT NULL
    );
IF OBJECT_ID('Dim_FechaAduana', 'U') IS NULL
    CREATE TABLE Dim_FechaAduana (
        id_FechaAduana INT PRIMARY KEY,
        Dia INT NULL,
        Mes INT NULL,
        Año INT NULL,
        Trimestre INT NULL
    );
    
IF OBJECT_ID('Dim_Bulto', 'U') IS NULL
    CREATE TABLE Dim_Bulto (
        id_bulto INT PRIMARY KEY,
        Clase_bulto VARCHAR(70)
    );    
    
    
IF OBJECT_ID('Dim_RecintoAduanero', 'U') IS NULL
    CREATE TABLE Dim_RecintoAduanero (
        id_Almacen INT PRIMARY KEY,
        RazonSocial VARCHAR(70),
        Jurisdiccion VARCHAR(25)
    );    
    
IF OBJECT_ID('Dim_EstadoMercaderia', 'U') IS NULL
    CREATE TABLE Dim_EstadoMercaderia (
        id_EstadoMercaderia INT PRIMARY KEY,
        estado_mercaderia VARCHAR(25)
    );    
    
    
IF OBJECT_ID('Dim_TransEmp', 'U') IS NULL
    CREATE TABLE Dim_TransEmp (
        id_ETRANS INT PRIMARY KEY,
        documento VARCHAR(12),
        razon_social VARCHAR(60)
    );    
    
IF OBJECT_ID('Dim_Aduana', 'U') IS NULL
      CREATE TABLE Dim_Aduana (
          id_Aduana INT PRIMARY KEY,
          Descripcion VARCHAR(25)
      );    
        
IF OBJECT_ID('Dim_Transporte', 'U') IS NULL
    CREATE TABLE Dim_Transporte (
        id_Transporte INT PRIMARY KEY,
        transporte VARCHAR(10)
    );    
    

IF OBJECT_ID('Dim_Puerto', 'U') IS NULL
    CREATE TABLE Dim_Puerto (
        id_Puerto INT PRIMARY KEY,
        nombre_puerto VARCHAR(35),
        nombre_pais VARCHAR(35)
    );    
    

IF OBJECT_ID('Dim_Mercaderia', 'U') IS NULL
    CREATE TABLE Dim_Mercaderia (
        id_Mercaderia INT PRIMARY KEY,
        mercaderia VARCHAR(90)
    ); 
    
IF OBJECT_ID('fact_importaciones', 'U') IS NULL
    CREATE TABLE fact_importaciones(
        id_FechaPuerto INT,
        id_FechaAduana INT,
        id_bulto INT,
        id_Almacen INT,
        id_EstadoMercaderia INT,
        id_ETRANS INT,
        id_Aduana INT,
        id_Transporte INT,
        id_Puerto INT,
        id_Mercaderia INT,
        PESO_NETO DECIMAL(10,2),
        PESO_BRUTO DECIMAL(10,2),
        CANTIDAD_BULTO INT,
        CANTIDAD_MERCANCIA INT,
        FOREIGN KEY (id_FechaPuerto) REFERENCES Dim_FechaPuerto(id_FechaPuerto),
        FOREIGN KEY (id_FechaAduana) REFERENCES Dim_FechaAduana(id_FechaAduana),
        FOREIGN KEY (id_bulto) REFERENCES Dim_Bulto(id_bulto),
        FOREIGN KEY (id_Almacen) REFERENCES Dim_RecintoAduanero(id_Almacen),
        FOREIGN KEY (id_EstadoMercaderia) REFERENCES Dim_EstadoMercaderia(id_EstadoMercaderia),
        FOREIGN KEY (id_ETRANS) REFERENCES Dim_TransEmp(id_ETRANS),
        FOREIGN KEY (id_Aduana) REFERENCES Dim_Aduana(id_Aduana),
        FOREIGN KEY (id_Transporte) REFERENCES Dim_Transporte(id_Transporte),
        FOREIGN KEY (id_Puerto) REFERENCES Dim_Puerto(id_Puerto),
        FOREIGN KEY (id_Mercaderia) REFERENCES Dim_Mercaderia(id_Mercaderia)
    );
""")
conn.commit()

# Poblado de tablas

In [3]:
def obtener_unicos_df(df, columna):
    """Devuelve un DataFrame con los valores únicos de una columna en el DataFrame."""
    if columna in df.columns:
        # Obtener los valores únicos y convertirlos en un DataFrame
        return pd.DataFrame(df[columna].unique(), columns=[columna])
    else:
        raise ValueError(f"La columna '{columna}' no existe en el DataFrame.")

In [7]:
Index(['CODI_ADUAN', 'NUME_CORRE', 'FECH_INGSI', 'TIPO_DOCUM', 'LIBR_TRIBU',
       'DNOMBRE', 'CODI_AGENT', 'FECH_LLEGA', 'VIA_TRANSP', 'EMPR_TRANS',
       'CODI_ALMA', 'FECH_RECEP', 'PAIS_ORIGE', 'PAIS_ADQUI', 'PUER_EMBAR',
       'FECH_EMBAR', 'NUME_SERIE', 'DESC_COMER', 'FLE_DOLAR', 'SEG_DOLAR',
       'PESO_NETO', 'PESO_BRUTO', 'UNID_FIQTY', 'UNID_FIDES', 'QUNICOM',
       'TUNICOM', 'SEST_MERCA', 'CANT_BULTO', 'CLASE', 'Razon_Social_Almacen',
       'Jurisdiccion_Almacen', 'Cpais', 'Dpais', 'Cpuerto', 'Dpuerto',
       'Descripcion_Aduana', 'Clase_Bulto', 'Razon Social', 'Codigo',
       'Estado_Mercancia', 'TRANSPORTE'],
      dtype='object')

NameError: name 'Index' is not defined

In [4]:
df_FechaPuerto=obtener_unicos_df(df,'FECH_INGSI')  # obtiene los nombres de distritos
df_FechaAduana=obtener_unicos_df(df,'FECH_LLEGA')  # obtiene los nombres de distritos
df_Aduana=obtener_unicos_df(df,'Descripcion_Aduana')  # obtiene los nombres de distritos
df_RecintoAduanero=df[['Razon_Social_Almacen','Jurisdiccion_Almacen']].drop_duplicates(ignore_index=True)
df_Mercaderia=obtener_unicos_df(df,'DESC_COMER')  # obtiene los nombres de distritos
df_Puerto=df[['Dpuerto','Dpais']].drop_duplicates(ignore_index=True)
df_Bulto=obtener_unicos_df(df,'Clase_Bulto')  # obtiene los nombres de distritos
df_Transporte=obtener_unicos_df(df,'TRANSPORTE')  # obtiene los nombres de distritos
df_EstadoMercaderia=obtener_unicos_df(df,'Estado_Mercancia')  # obtiene los nombres de distritos
df_TransEmp=df[['LIBR_TRIBU','Razon Social']].drop_duplicates(ignore_index=True)


# Función para agregar identificadores únicos
def agregar_identificador(df, nombre_id):
    df[nombre_id] = pd.Series(range(1, len(df) + 1))
    return df

# Aplicar a varios DataFrames
df_FechaPuerto = agregar_identificador(df_FechaPuerto, 'id_fechaPuerto')
df_FechaAduana = agregar_identificador(df_FechaAduana, 'id_fechaAduana')
df_Aduana = agregar_identificador(df_Aduana, 'id_aduana')
df_RecintoAduanero = agregar_identificador(df_RecintoAduanero, 'id_recinto')
df_Mercaderia = agregar_identificador(df_Mercaderia, 'id_mercaderia')
df_Puerto = agregar_identificador(df_Puerto, 'id_puerto')
df_Bulto = agregar_identificador(df_Bulto, 'id_bulto')
df_Transporte = agregar_identificador(df_Transporte, 'id_transporte')
df_EstadoMercaderia = agregar_identificador(df_EstadoMercaderia, 'id_estadoMercaderia')
df_TransEmp = agregar_identificador(df_TransEmp, 'id_transEmp')

In [5]:
df_FechaPuerto['FECH_INGSI'] = pd.to_datetime(df_FechaPuerto['FECH_INGSI'], format='%Y-%m-%d')

In [18]:
try:
    for index, row in df_FechaPuerto.iterrows():
        cursor.execute(
            "INSERT INTO Dim_FechaPuerto (id_FechaPuerto, Dia, Mes, Año, Trimestre) VALUES (?, ?, ?, ?, ?)",
            row['id_fechaPuerto'], 
            row['FECH_INGSI'].day, 
            row['FECH_INGSI'].month, 
            row['FECH_INGSI'].year, 
            row['FECH_INGSI'].quarter
        )
    conn.commit()
except Exception as e:
    print(f"Error al insertar registros en Dim_FechaPuerto: {e}")

In [9]:
from tqdm import tqdm
try:
    for index, row in df_FechaPuerto.iterrows():
        cursor.execute(
            "INSERT INTO Dim_FechaPuerto (id_FechaPuerto, Dia, Mes, Año, Trimestre) VALUES (?, ?, ?, ?, ?)",
            row['id_fechaPuerto'], 
            row['FECH_INGSI'].day, 
            row['FECH_INGSI'].month, 
            row['FECH_INGSI'].year, 
            row['FECH_INGSI'].quarter
        )
    conn.commit()
except Exception as e:
    print(f"Error al insertar registros en Dim_FechaPuerto: {e}")
try:
    for index, row in tqdm(df_FechaAduana.iterrows()):
        # Convertir a tipos nativos de Python (int o None)
        id_fechaAduana = int(row['id_fechaAduana']) if not pd.isnull(row['id_fechaAduana']) else None
        if row['FECH_LLEGA'] == 0:
            dia, mes, año, trimestre = 0, 0, 0, 0
        else:
            fecha = pd.to_datetime(str(row['FECH_LLEGA']), format='%Y%m%d')
            dia, mes, año, trimestre = fecha.day, fecha.month, fecha.year, fecha.quarter
        
        # Ejecutar la inserción
        cursor.execute(
            "INSERT INTO Dim_FechaAduana (id_FechaAduana, Dia, Mes, Año, Trimestre) VALUES (?, ?, ?, ?, ?)",
            id_fechaAduana, dia, mes, año, trimestre
        )
    conn.commit()
except Exception as e:
    print(f"Error al insertar registros en Dim_FechaAduana: {e}")

try:
    for index, row in df_Bulto.iterrows():
        cursor.execute(
            "INSERT INTO Dim_Bulto (id_bulto, Clase_bulto) VALUES (?, ?)",
            row['id_bulto'], 
            row['Clase_Bulto']
        )
    conn.commit()
except Exception as e:
    print(f"Error al insertar registros en Dim_Bulto: {e}")
try:
    for index, row in df_RecintoAduanero.iterrows():
        cursor.execute(
            "INSERT INTO Dim_RecintoAduanero (id_Almacen, RazonSocial, Jurisdiccion) VALUES (?, ?, ?)",
            row['id_recinto'], 
            row['Razon_Social_Almacen'], 
            row['Jurisdiccion_Almacen']
        )
    conn.commit()
except Exception as e:
    print(f"Error al insertar registros en Dim_RecintoAduanero: {e}")
try:
    for index, row in df_EstadoMercaderia.iterrows():
        cursor.execute(
            "INSERT INTO Dim_EstadoMercaderia (id_EstadoMercaderia, estado_mercaderia) VALUES (?, ?)",
            row['id_estadoMercaderia'], 
            row['Estado_Mercancia']
        )
    conn.commit()
except Exception as e:
    print(f"Error al insertar registros en Dim_EstadoMercaderia: {e}")
try:
    for index, row in df_TransEmp.iterrows():
        cursor.execute(
            "INSERT INTO Dim_TransEmp (id_ETRANS, documento, razon_social) VALUES (?, ?, ?)",
            row['id_transEmp'], 
            row['LIBR_TRIBU'], 
            row['Razon Social']
        )
    conn.commit()
except Exception as e:
    print(f"Error al insertar registros en Dim_TransEmp: {e}")
try:
    for index, row in df_Aduana.iterrows():
        cursor.execute(
            "INSERT INTO Dim_Aduana (id_Aduana, Descripcion) VALUES (?, ?)",
            row['id_aduana'], 
            row['Descripcion_Aduana']
        )
    conn.commit()
except Exception as e:
    print(f"Error al insertar registros en Dim_Aduana: {e}")
try:
    for index, row in df_Transporte.iterrows():
        cursor.execute(
            "INSERT INTO Dim_Transporte (id_Transporte, transporte) VALUES (?, ?)",
            row['id_transporte'], 
            row['TRANSPORTE']
        )
    conn.commit()
except Exception as e:
    print(f"Error al insertar registros en Dim_Transporte: {e}")
try:
    for index, row in df_Puerto.iterrows():
        cursor.execute(
            "INSERT INTO Dim_Puerto (id_Puerto, nombre_puerto, nombre_pais) VALUES (?, ?, ?)",
            row['id_puerto'], 
            row['Dpuerto'], 
            row['Dpais']
        )
    conn.commit()
except Exception as e:
    print(f"Error al insertar registros en Dim_Puerto: {e}")
try:
    for index, row in df_Mercaderia.iterrows():
        cursor.execute(
            "INSERT INTO Dim_Mercaderia (id_Mercaderia, mercaderia) VALUES (?, ?)",
            row['id_mercaderia'], 
            row['DESC_COMER']
        )
    conn.commit()
except Exception as e:
    print(f"Error al insertar registros en Dim_Mercaderia: {e}")

Error al insertar registros en Dim_FechaPuerto: 'str' object has no attribute 'day'


673it [00:00, 2791.37it/s]
1237928it [02:54, 7075.98it/s]


In [7]:
# Merge con cada dimensión para obtener los IDs
df = df.merge(df_FechaPuerto[['FECH_INGSI', 'id_fechaPuerto']], on='FECH_INGSI', how='left')
df = df.merge(df_FechaAduana[['FECH_LLEGA', 'id_fechaAduana']], on='FECH_LLEGA', how='left')
df = df.merge(df_Bulto[['Clase_Bulto', 'id_bulto']], on='Clase_Bulto', how='left')
df = df.merge(df_RecintoAduanero[['Razon_Social_Almacen', 'Jurisdiccion_Almacen', 'id_recinto']],
              on=['Razon_Social_Almacen', 'Jurisdiccion_Almacen'], how='left')
df = df.merge(df_EstadoMercaderia[['Estado_Mercancia', 'id_estadoMercaderia']], on='Estado_Mercancia', how='left')
df = df.merge(df_TransEmp[['LIBR_TRIBU', 'Razon Social', 'id_transEmp']],
              on=['LIBR_TRIBU', 'Razon Social'], how='left')
df = df.merge(df_Aduana[['Descripcion_Aduana', 'id_aduana']], on='Descripcion_Aduana', how='left')
df = df.merge(df_Transporte[['TRANSPORTE', 'id_transporte']], on='TRANSPORTE', how='left')
df = df.merge(df_Puerto[['Dpuerto', 'Dpais', 'id_puerto']], on=['Dpuerto', 'Dpais'], how='left')
df = df.merge(df_Mercaderia[['DESC_COMER', 'id_mercaderia']], on='DESC_COMER', how='left')


In [6]:
df['FECH_INGSI'] = pd.to_datetime(df['FECH_INGSI'], format='%Y-%m-%d')

In [17]:
# Preparar los datos para la inserción
data_to_insert = df[[
    'id_fechaPuerto', 'id_fechaAduana', 'id_bulto', 'id_recinto', 'id_estadoMercaderia',
    'id_transEmp', 'id_aduana', 'id_transporte', 'id_puerto', 'id_mercaderia',
    'PESO_NETO', 'PESO_BRUTO', 'CANT_BULTO', 'UNID_FIQTY'
]].values.tolist()
from tqdm import tqdm

# Dividir los datos en lotes (ejemplo: 1000 registros por lote)
batch_size = 1000
batches = [data_to_insert[i:i + batch_size] for i in range(0, len(data_to_insert), batch_size)]

# Insertar los datos en lotes con barra de progreso
try:
    for batch in tqdm(batches, desc="Insertando registros en fact_importaciones"):
        cursor.executemany("""
            INSERT INTO fact_importaciones (
                id_FechaPuerto, id_FechaAduana, id_bulto, id_Almacen, id_EstadoMercaderia, id_ETRANS, id_Aduana, id_Transporte, id_Puerto, id_Mercaderia,
                PESO_NETO, PESO_BRUTO, CANTIDAD_BULTO, CANTIDAD_MERCANCIA
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, batch)
        conn.commit()
    print("Datos insertados correctamente en fact_importaciones")
except Exception as e:
    print(f"Error al insertar registros en fact_importaciones: {e}")

Insertando registros en fact_importaciones: 100%|██████████████████████████████████| 6676/6676 [14:26<00:00,  7.71it/s]

Datos insertados correctamente en fact_importaciones


In [18]:
cursor.close()
conn.close()


# Tratamiento de datos adicional

In [16]:
import pandas as pd
import pyodbc
import numpy as np

#----------------------------------- Configura la conexión a SQL Server DataMart --------------------------------
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'  # Asegúrate de tener el controlador adecuado
    'SERVER=localhost\SQLEXPRESS01;'                  # Reemplaza con el nombre de tu servidor
    'DATABASE=datamart_importaciones_2;'           # Reemplaza con el nombre de tu base de datos
    'Trusted_Connection=yes;'                           # Reemplaza con tu contraseña
)
cursor = conn.cursor()


In [27]:
try:
    for index, row in tqdm(df.iterrows()):
        # Obtener los IDs relacionados de las dimensiones
        id_fechaPuerto = int(df_FechaPuerto.loc[df_FechaPuerto['FECH_INGSI'] == row['FECH_INGSI'], 'id_fechaPuerto'].values[0])
        id_fechaAduana = int(df_FechaAduana.loc[df_FechaAduana['FECH_LLEGA'] == row['FECH_LLEGA'], 'id_fechaAduana'].values[0])
        id_bulto = int(df_Bulto.loc[df_Bulto['Clase_Bulto'] == row['Clase_Bulto'], 'id_bulto'].values[0])
        id_recinto = int(df_RecintoAduanero.loc[
            (df_RecintoAduanero['Razon_Social_Almacen'] == row['Razon_Social_Almacen']) &
            (df_RecintoAduanero['Jurisdiccion_Almacen'] == row['Jurisdiccion_Almacen']),
            'id_recinto'
        ].values[0])
        id_estadoMercaderia = int(df_EstadoMercaderia.loc[df_EstadoMercaderia['Estado_Mercancia'] == row['Estado_Mercancia'], 'id_estadoMercaderia'].values[0])
        id_transEmp = int(df_TransEmp.loc[
            (df_TransEmp['LIBR_TRIBU'] == row['LIBR_TRIBU']) &
            (df_TransEmp['Razon Social'] == row['Razon Social']),
            'id_transEmp'
        ].values[0])
        id_aduana = int(df_Aduana.loc[df_Aduana['Descripcion_Aduana'] == row['Descripcion_Aduana'], 'id_aduana'].values[0])
        id_transporte = int(df_Transporte.loc[df_Transporte['TRANSPORTE'] == row['TRANSPORTE'], 'id_transporte'].values[0])
        id_puerto = int(df_Puerto.loc[
            (df_Puerto['Dpuerto'] == row['Dpuerto']) & (df_Puerto['Dpais'] == row['Dpais']),
            'id_puerto'
        ].values[0])
        id_mercaderia = int(df_Mercaderia.loc[df_Mercaderia['DESC_COMER'] == row['DESC_COMER'], 'id_mercaderia'].values[0])

        # Insertar en la tabla de hechos
        cursor.execute(
            """
            INSERT INTO fact_importaciones (
                id_FechaPuerto, id_FechaAduana, id_bulto, id_Almacen, id_EstadoMercaderia, id_ETRANS, id_Aduana, id_Transporte, id_Puerto, id_Mercaderia,
                PESO_NETO, PESO_BRUTO, CANTIDAD_BULTO, CANTIDAD_MERCANCIA
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """,
            id_fechaPuerto,
            id_fechaAduana,
            id_bulto,
            id_recinto,
            id_estadoMercaderia,
            id_transEmp,
            id_aduana,
            id_transporte,
            id_puerto,
            id_mercaderia,
            row['PESO_NETO'],
            row['PESO_BRUTO'],
            row['CANT_BULTO'],
            row['UNID_FIQTY']
        )
    conn.commit()
except Exception as e:
    print(f"Error al insertar registros en fact_importaciones: {e}")


1239it [02:10,  9.48it/s]


KeyboardInterrupt: 

In [28]:
cursor.close()
conn.close()

In [24]:
df['CANT_BULTO'] = df['CANT_BULTO'].astype(int)
df['UNID_FIQTY'] = df['UNID_FIQTY'].astype(int)
df['PESO_NETO'] = df['PESO_NETO'].astype(float)
df['PESO_BRUTO'] = df['PESO_BRUTO'].astype(float)

In [4]:
# Tabla de ImportacionB (ImportacionB)

cursor.execute("""
IF OBJECT_ID('ImportacionB', 'U') IS NULL
BEGIN
    CREATE TABLE ImportacionB (
        CODI_ADUAN VARCHAR(3) NOT NULL, -- Código de aduana
        ANO_PRESE VARCHAR(2) NOT NULL, -- Año de numeración de la DUA
        NUME_CORRE VARCHAR(6) NOT NULL, -- Número de DUA
        NUME_SECUP NUMERIC(4), -- Número de secuencia del proveedor
        FECH_INGSI NUMERIC(8), -- Fecha de ingreso de la póliza
        CODI_PROVE VARCHAR(10), -- Código del proveedor
        NOMB_PROVE VARCHAR(60), -- Nombre del proveedor
        PART_NANDI NUMERIC(10), -- Partida arancelaria
        NUME_ITEM VARCHAR(4), -- Número de serie
        VFOB_ITEM NUMERIC(17,6), -- FOB en dólares (FOB unitario x cantidad)
        NOMB_COMER VARCHAR(30), -- Nombre comercial
        MARC_COMER VARCHAR(25), -- Marca comercial
        MODE_MERCD VARCHAR(30), -- Modelo
        CANT_MERCD NUMERIC(17,6), -- Cantidad
        UNID_MERCD VARCHAR(3), -- Unidad de medida
        CARA_TIPO VARCHAR(90), -- Característica del producto
        PAIS_ORIGE VARCHAR(2), -- País de origen
        ARO_ANO VARCHAR(4), -- Año de producción
        ESTA_MERCD VARCHAR(2), -- Estado del producto
        VFOB_UNITA NUMERIC(17,6) -- FOB unitario
    );
END
""")

conn.commit()


In [18]:
# Tabla de Exportacion (Exportaciones)

cursor.execute("""
IF OBJECT_ID('Exportaciones', 'U') IS NULL
BEGIN
    CREATE TABLE Exportaciones (
        CADU VARCHAR(3) NOT NULL, -- Código de aduana
        FANO VARCHAR(4) NOT NULL, -- Fecha de numeración de la orden de embarque
        NDCL VARCHAR(6) NOT NULL, -- Número de orden de embarque
        FNUM NUMERIC(8), -- Fecha de numeración de la orden de embarque
        FEMB NUMERIC(8), -- Fecha de embarque
        FECH_RECEP NUMERIC(8), -- Fecha de recepción de la declaración
        NDCLREG VARCHAR(6) NOT NULL, -- Número de la declaración de exportación
        FREG NUMERIC, -- Fecha de regularización
        FANOREG VARCHAR(4), -- Año de regularización de exportación
        CAGE VARCHAR(4), -- Código de agente de aduana
        TDOC VARCHAR(1), -- Tipo de documento del exportador
        NDOC VARCHAR(11), -- Número de documento del exportador
        DNOMBRE VARCHAR(40), -- Razón social del exportador
        CPAIDES VARCHAR(3), -- Código de país de destino
        CPUEDES VARCHAR(6), -- Código del puerto de destino
        CVIATRA VARCHAR(1), -- Código de la vía de transporte
        CUNITRA VARCHAR(1), -- Código de unidad de transporte
        CEMPTRA VARCHAR(4), -- Código de empresa de transporte
        DMAT VARCHAR(30), -- Matrícula de la nave
        NCON VARCHAR(20), -- Número de conocimiento
        CENTFIN VARCHAR(3), -- Código de entidad financiera
        CALM VARCHAR(4), -- Código de almacén
        DNOMPRO VARCHAR(60), -- Nombre del proveedor
        DDIRPRO VARCHAR(80), -- Dirección del proveedor
        DK VARCHAR(1), -- Indicador de teledespacho para O.E.
        DK2 VARCHAR(1), -- Indicador de teledespacho para DUE
        NSER VARCHAR(4) NOT NULL, -- Número de serie
        PART_NANDI NUMERIC(10), -- Código de la partida nandina
        DCOM VARCHAR(100), -- Descripción comercial
        DMER2 VARCHAR(100), -- Descripción comercial
        DMER3 VARCHAR(100), -- Descripción comercial
        DMER4 VARCHAR(100), -- Descripción comercial
        DMER5 VARCHAR(100), -- Descripción comercial
        VFOBSERDOL NUMERIC(11,3), -- Valor FOB de la serie
        VPESNET NUMERIC(14,3), -- Peso neto de la serie
        VPESBRU NUMERIC(14,3), -- Peso bruto de la serie
        QUNIFIS NUMERIC(14,3), -- Cantidad exportada
        TUNIFIS VARCHAR(10), -- Unidad de medida
        QUNICOM NUMERIC(14,3), -- Cantidad de unidad comercial
        TUNICOM VARCHAR(3), -- Tipo de unidad comercial
        UBIGEO VARCHAR(6) -- Código de ubicación geográfica
    );
END
""")

conn.commit()


Las siguientes son tablas que contienen los recursos para algunas abreviaciones (paises, puertos, etc.)

In [ ]:
# Tablas adicionales 

cursor.execute("""
IF OBJECT_ID('exp', 'U') IS NULL
BEGIN
    CREATE TABLE exp (
        CODI_ADUAN VARCHAR(3) NOT NULL,
        ANO_PRESE VARCHAR(2) NOT NULL,
        NUME_CORRE VARCHAR(6) NOT NULL,
        FECH_INGSI NUMERIC(8),
        TIPO_DOCUM VARCHAR(1),
        LIBR_TRIBU VARCHAR(11),
    );
END
""")

conn.commit()


## Inserción de información

### ImportacionesA

Primero realizaremos un pre procesamiento de la información breve, Luego el llenado de las tablas

In [10]:
# Cargamos los datos de los archivos ma e ImportacionA (mismo contenido)
df_ma = pd.read_csv('ImportacionA.csv',encoding='utf-8', sep=';')
df_mam = pd.read_csv('mam.csv',encoding='utf-8', sep=';')

In [11]:
# Unimos ambas tablas en una unica de importaciones 
df_importaciones=pd.concat([df_ma,df_mam],axis=0)
df_importaciones.shape

(7152794, 58)

In [12]:
print("Campos con nan")
for c in df_importaciones.columns:
    x=sum(df_importaciones[c].isna())
    if x!=0:
        print(c, x)

Campos con nan
DNOMBRE 6
EMPR_TRANS 610774
BANC_CANCE 2754908
CODI_ENFIN 5297838
PAIS_ORIGE 3
PUER_EMBAR 576
DESC_MATCO 389720
DESC_USOAP 681832
DESC_FOPRE 40817
DESC_OTROS 1609172
IMPR_RELIQ 6117348


In [13]:
# Eliminamos duplicados y reemplazamos nan (ocasionan problemas al momento de subir la base)
df_importaciones=df_importaciones.drop_duplicates()

ValueError: Must specify a fill 'value' or 'method'.

In [14]:
df_importaciones = df_importaciones.where(pd.notnull(df_importaciones), None)

In [141]:
df_almacen = pd.read_fwf('RecintAduaner.txt', widths=[7, 72, 50, 30, 15, 50, 10, 10])
df_almacen.drop_duplicates(subset="Codigo",inplace=True)
df_aver = pd.merge(df_importaciones, df_almacen[['Codigo','Razon Social',"Jurisdiccion"]], right_on="Codigo",left_on="CODI_ALMA", how="left")
df_aver['Razon Social'] = df_aver['Razon Social'].fillna('No mencionado')
df_aver['Codigo'] = df_aver['Codigo'].fillna(10000)
df_aver['CODI_ALMA'] = df_aver['Codigo']
df_aver['Jurisdiccion'] = df_aver['Jurisdiccion'].fillna("No mencionado")
df_aver.drop(columns='Codigo',inplace=True)
df_aver.rename(columns={"Razon Social":"Razon_Social_Almacen","Jurisdiccion":"Jurisdiccion_Almacen"})
df_almacen.loc[581] = [10000,"No mencionado","No mencionado",0,0,0,0,0]

In [152]:
df_aver.to_csv("pasito.csv",index=False,sep=',')

In [12]:
import pandas as pd
df_aver = pd.read_csv("pasito.csv",sep=',')

C:\Users\max_1\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (4,5,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
df_puerto = pd.read_fwf("Puertos.txt", widths=[6, 50, 7, 51, 15, 50, 10, 10,10])

In [16]:
df_aver = pd.merge(df_aver, df_puerto[['Cpais','Dpais','Cpuerto','Dpuerto',"Codigo_completo"]], how='left', left_on='PUER_EMBAR', right_on='Codigo_completo')

In [19]:
df_aver.drop(columns='PUER_EMBAR_3D',inplace=True)

In [14]:
df_puerto['Codigo_completo'] = df_puerto['Cpais'] + df_puerto['Cpuerto']

In [32]:
df_aver['Cpais'] = df_aver['Cpais'].fillna(0)
df_aver['Dpais'] = df_aver['Dpais'].fillna("No mencionado")
df_aver['Cpuerto'] = df_aver['Cpuerto'].fillna(0)
df_aver['Dpuerto'] = df_aver['Dpuerto'].fillna("No mencionado")
df_aver['Codigo_completo'] = df_aver['Codigo_completo'].fillna(0)
df_aver['PUER_EMBAR'] = df_aver['Codigo_completo']
df_aver.drop(columns='Codigo_completo',inplace=True)

In [33]:
df_puerto.loc[35108] = [0,"No mencionado",0,"No mencionado",0,0,0,0,0,0]

In [36]:
df_backup = df_aver.copy()

In [39]:
df_aduanas = pd.read_fwf("Aduanas.txt", widths=[7, 71, 7, 51, 15, 50])

In [44]:
df_aver = pd.merge(df_aver, df_aduanas[["Codigo","Descripcion"]], how='left', left_on='CODI_ADUAN', right_on='Codigo')

In [48]:
df_aver.drop(columns='Codigo',inplace=True)

In [50]:
df_aver.rename(columns={'Descripcion':"Descripcion_Aduana"},inplace=True)

In [2]:
import pandas as pd 
import gc

In [3]:
df_aver = pd.read_csv("mergeado.csv")

C:\Users\max_1\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (4,5,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df_aver['CLASE'].unique()

array(['CAJ', 'CRT', 'BUL', 'BOB', 'Z1', 'PK', 'PAL', 'PQT', 'SAC', 'BLS',
       'FAR', 'GRN', 'VEI', 'PZS', 'CL', 'CS', 'BG', 'RLL', 'CT', 'DR',
       'CTN', 'TAM', 'BN', 'LOT', 'BL', 'HUA', 'UNI', 'SKI', 'BX', 'BE',
       'RL', 'JC', 'CIL', 'KGS', 'RO', 'BAL', 'PL', 'CNT', 'BRR', 'CR',
       'BU', 'TNM', 'TK', 'BOT', 'CJ', 'JAB', 'KG', 'JGS', 'LAT', 'BI',
       'ATD', 'BLL', 'BID', 'FR', 'EST', 'CU', 'BS', 'TY', 'ST', 'CA',
       'BD', 'RT', 'ESQ', 'PI', 'BA', 'GB', 'PA', 'GAL', 'JY', 'PAR',
       'PZ'], dtype=object)

In [19]:
df_bulto = pd.read_fwf("UnidMercancia.txt",widths=[7, 63, 7, 51, 15, 50])
df_bulto = df_bulto[['Codigo','Descripcion']]

In [20]:
df_aver = pd.merge(df_aver, df_bulto, how='left', left_on='CLASE', right_on='Codigo')

In [23]:
df_aver.rename(columns={'Descripcion':'Clase_Bulto'},inplace=True)

In [27]:
df_aver.to_csv("df_final.csv",index=False,sep=',')

In [3]:
import pandas as pd
import gc

In [4]:
df_aver = pd.read_csv("df_final.csv")

C:\Users\max_1\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (4,5,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [222]:
columnas_con_na = df_final.columns[df_final.isnull().any()].tolist()
print("Columnas con valores nulos:")
print(columnas_con_na)

Columnas con valores nulos:
['DNOMBRE', 'PAIS_ORIGE']


In [181]:
df_final['SEST_MERCA'].unique()

array([10, 20, 12, 14, 28, 22, 99, 16, 26, 17, 27, 24], dtype=int64)

In [221]:
df_final.drop(columns=['DESC_MATCO','DESC_USOAP','DESC_FOPRE','DESC_OTROS'],inplace=True)

In [224]:
df_final.dropna(inplace=True)

In [227]:
df_final.to_csv("df_recontrafinal.csv",index=False)

In [179]:
df_final['DESC_OTROS'].isna().sum()

1505953

In [175]:
df_final.drop(columns='Codigo',inplace=True)

In [180]:
df_final.columns

Index(['CODI_ADUAN', 'NUME_CORRE', 'FECH_INGSI', 'TIPO_DOCUM', 'LIBR_TRIBU',
       'DNOMBRE', 'CODI_AGENT', 'FECH_LLEGA', 'VIA_TRANSP', 'EMPR_TRANS',
       'CODI_ALMA', 'FECH_RECEP', 'PAIS_ORIGE', 'PAIS_ADQUI', 'PUER_EMBAR',
       'FECH_EMBAR', 'NUME_SERIE', 'DESC_COMER', 'DESC_MATCO', 'DESC_USOAP',
       'DESC_FOPRE', 'DESC_OTROS', 'FLE_DOLAR', 'SEG_DOLAR', 'PESO_NETO',
       'PESO_BRUTO', 'UNID_FIQTY', 'UNID_FIDES', 'QUNICOM', 'TUNICOM',
       'SEST_MERCA', 'CANT_BULTO', 'CLASE', 'Razon_Social_Almacen',
       'Jurisdiccion_Almacen', 'Cpais', 'Dpais', 'Cpuerto', 'Dpuerto',
       'Descripcion_Aduana', 'Clase_Bulto', 'Razon Social'],
      dtype='object')

In [204]:
df_mercancia = pd.read_fwf("EstMercancia.txt",widths=[8,63,10,10,10,10])
df_mercancia = df_mercancia[['Codigo','Descripcion']]

In [207]:
df_final.columns

Index(['CODI_ADUAN', 'NUME_CORRE', 'FECH_INGSI', 'TIPO_DOCUM', 'LIBR_TRIBU',
       'DNOMBRE', 'CODI_AGENT', 'FECH_LLEGA', 'VIA_TRANSP', 'EMPR_TRANS',
       'CODI_ALMA', 'FECH_RECEP', 'PAIS_ORIGE', 'PAIS_ADQUI', 'PUER_EMBAR',
       'FECH_EMBAR', 'NUME_SERIE', 'DESC_COMER', 'DESC_MATCO', 'DESC_USOAP',
       'DESC_FOPRE', 'DESC_OTROS', 'FLE_DOLAR', 'SEG_DOLAR', 'PESO_NETO',
       'PESO_BRUTO', 'UNID_FIQTY', 'UNID_FIDES', 'QUNICOM', 'TUNICOM',
       'SEST_MERCA', 'CANT_BULTO', 'CLASE', 'Razon_Social_Almacen',
       'Jurisdiccion_Almacen', 'Cpais', 'Dpais', 'Cpuerto', 'Dpuerto',
       'Descripcion_Aduana', 'Clase_Bulto', 'Razon Social'],
      dtype='object')

In [211]:
df_final = pd.merge(df_final,df_mercancia,left_on="SEST_MERCA",right_on='Codigo',how='left')

In [218]:
df_final.rename(columns={'Descripcion':'Estado_Mercancia'},inplace=True)

In [219]:
df_final.head()

,CODI_ADUAN,NUME_CORRE,FECH_INGSI,TIPO_DOCUM,LIBR_TRIBU,DNOMBRE,CODI_AGENT,FECH_LLEGA,VIA_TRANSP,EMPR_TRANS,...,Jurisdiccion_Almacen,Cpais,Dpais,Cpuerto,Dpuerto,Descripcion_Aduana,Clase_Bulto,Razon Social,Codigo,Estado_Mercancia
0,235,135859,20240903,4,20606352825,2M DIESEL S.A.C.,14,20240825,4,UX,...,A NIVEL NACIONAL,TR,TURKEY,IST,ISTANBUL,AEREA Y POSTAL EX-IAAC,BULTOS ...,AIR EUROPA LINEAS AEREAS S.A. SUCURSAL DEL PERU,10,NUEVO/BUENO
1,235,135859,20240903,4,20606352825,2M DIESEL S.A.C.,14,20240825,4,UX,...,A NIVEL NACIONAL,TR,TURKEY,IST,ISTANBUL,AEREA Y POSTAL EX-IAAC,BULTOS ...,AIR EUROPA LINEAS AEREAS S.A. SUCURSAL DEL PERU,10,NUEVO/BUENO
2,235,137787,20240906,4,20503101913,A & N PROYECTOS S.A.C. - A & N S.A.C.,3728,20240907,4,AF,...,No mencionado,ES,SPAIN,BCN,BARCELONA,AEREA Y POSTAL EX-IAAC,BULTOS ...,SOCIETE AIR FRANCE SUCURSAL EN EL PERU,10,NUEVO/BUENO
3,235,137787,20240906,4,20503101913,A & N PROYECTOS S.A.C. - A & N S.A.C.,3728,20240907,4,AF,...,No mencionado,ES,SPAIN,BCN,BARCELONA,AEREA Y POSTAL EX-IAAC,BULTOS ...,SOCIETE AIR FRANCE SUCURSAL EN EL PERU,10,NUEVO/BUENO
4,235,137787,20240906,4,20503101913,A & N PROYECTOS S.A.C. - A & N S.A.C.,3728,20240907,4,AF,...,No mencionado,ES,SPAIN,BCN,BARCELONA,AEREA Y POSTAL EX-IAAC,BULTOS ...,SOCIETE AIR FRANCE SUCURSAL EN EL PERU,10,NUEVO/BUENO


In [78]:
def transformar_valor(valor):
    # Verificar si es un número flotante y termina en .0
    if isinstance(valor, float) and valor.is_integer():
        # Convertir a entero, luego a string de longitud 4 con ceros a la izquierda
        return f"{int(valor):04d}"
    # Verificar si es un string que termina con .0
    elif isinstance(valor, str) and valor.endswith('.0') and valor[:-2].isdigit():
        # Convertir a entero, luego a string de longitud 4 con ceros a la izquierda
        return f"{int(float(valor)) :04d}"
    # Dejar los valores no numéricos tal cual
    return str(valor)  # Convertir todo lo demás a string

In [83]:
df_aver = df_aver.loc[~df_aver['EMPR_TRANS'].isin(['nan'])]

In [85]:
df_empres = pd.read_fwf("LinAereasManif.txt", widths=[7, 71, 7, 51, 15, 50])

In [86]:
df_empres
df_empres['Codigo'] = df_empres['Codigo'].astype(str)

In [112]:
df_empres = df_empres[['Codigo','Descripcion']]
df_empres.rename(columns={'Descripcion':'Razon Social'},inplace=True)

In [114]:
df_empres

,Codigo,Razon Social
0,0A,SERVICIOS AEREOS COLIBRI S.A.
1,0B,AERO CONDOR S.A.
2,0C,LAN PERU S.A
3,0D,AVIACION LIDER S.A.
4,0E,TRANS AMERICAN AIR LINES S.A.
...,...,...
128,XX,TAM LINEAS AEREAS S.A.
129,YU,AEROLINEAS DOMINICANAS S.A. (DOMINAIR)
130,ZA,BAY AIR CARGO SA
131,ZS,ANGLO AMERICAN CORPORATION OF SOUTH AFRICA LTD.


In [115]:
df_empres_2 = pd.read_fwf("EmpNacional.txt",widths=[7, 82,15, 51, 15, 50])
df_empres_2.drop_duplicates(subset='Codigo',inplace=True)
df_empres_2['Razon Social'] = df_empres_2['Razon Social'].str.replace(r'\bRUC\b|\d+', '', regex=True).str.strip()
df_empres_2['Codigo'] = df_empres_2['Codigo'].astype(str)

In [116]:
df_empres_2 = df_empres_2[['Codigo',"Razon Social"]]

In [117]:
df_empres_2.drop(2,axis=0,inplace=True)

In [118]:
df_empres_2

,Codigo,Razon Social
0,nan,INTEROCEANICA POMA (PERMISO OCASIONAL)
3,0000,TRANSPORTES EMELINA MAGDALENA ROMERO RIVAS EIRL
4,0001,AMERICA INTERNACIONAL TOURS S.A. AMINTOURS
6,0002,EXPRESO INTERNACIONAL ORMEÑO S.A.
8,0003,TRANSPORTES VILLANUEVA S.A (TRANVINSA)
...,...,...
5072,9960,LUPACA RAMOS WILFREDO
5073,9993,PARTICULAR
5075,9997,TRABAJOS MARITIMOS S.A.
5076,9998,PROVISIONAL


In [119]:
df_empres_3 = pd.read_fwf("AgenMar.txt",widths=[7, 71,15, 51, 15, 50])
df_empres_3.drop_duplicates(subset='Codigo',inplace=True)
df_empres_3['Agente_Maritimo'] = df_empres_3['Agente_Maritimo'].str.replace(r'\bRUC\b|\d+', '', regex=True).str.strip()
df_empres_3['Codigo'] = df_empres_3['Codigo'].astype(str)
df_empres_3.rename(columns={"Agente_Maritimo":"Razon Social"},inplace=True)
df_empres_3 = df_empres_3[['Codigo','Razon Social']]

In [120]:
df_empres_3['Codigo'] = df_empres_3['Codigo'].apply(lambda x: f"{int(x):04d}")


In [121]:
df_empres_3

,Codigo,Razon Social
0,0019,ABRAHAM WOLL DAVILA S.A.
1,0028,SANTA SOFIA PUERTOS S.A.
3,0037,AGENCIA MARITIMA COLUMBIA DEL PERU S.A.
4,0046,AGENCIA MARITIMA SAN CARLOS S.A.
5,0055,MAR PISCO S R LTDA
...,...,...
520,9957,AGENCIA MARITIMA FULL SEAS S.A.
521,9966,LATINOAMERICANA INVERSIONES MARITIMAS S.
523,9975,AGENCIA MARITIMA FULL SHIPPING SERVICE S
524,9984,AGENCIA MARITIMA MARKO BUSONICH S.R.L.


In [122]:
dataframes = {
    4: df_empres,
    7: df_empres_2,
    1: df_empres_3,
}

In [123]:
from tqdm import tqdm

chunk_size = 100000  # Ajusta este valor según la memoria disponible
num_chunks = len(df_aver) // chunk_size + 1

merged_chunks = []

for i in tqdm(range(num_chunks)):
    # Seleccionar un fragmento del DataFrame
    chunk = df_aver.iloc[i * chunk_size:(i + 1) * chunk_size]

    # Filtrar y hacer merge por cada valor de `via_transporte` en el fragmento
    for transporte, df_empresas in dataframes.items():
        df_filtered = chunk[chunk['VIA_TRANSP'] == transporte]
        merged_chunk = pd.merge(df_filtered, df_empresas, how='left', left_on='EMPR_TRANS',right_on='Codigo')
        merged_chunks.append(merged_chunk)

# Combina solo los fragmentos procesados
df_final = pd.concat(merged_chunks, ignore_index=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 67/67 [00:34<00:00,  1.96it/s]


In [144]:
df_final['Razon Social'] = df_final['Razon Social'].fillna("No mencionado")

In [145]:
df_final.shape

(6675814, 43)

In [146]:
df_final.columns

Index(['CODI_ADUAN', 'NUME_CORRE', 'FECH_INGSI', 'TIPO_DOCUM', 'LIBR_TRIBU',
       'DNOMBRE', 'CODI_AGENT', 'FECH_LLEGA', 'VIA_TRANSP', 'EMPR_TRANS',
       'CODI_ALMA', 'FECH_RECEP', 'PAIS_ORIGE', 'PAIS_ADQUI', 'PUER_EMBAR',
       'FECH_EMBAR', 'NUME_SERIE', 'DESC_COMER', 'DESC_MATCO', 'DESC_USOAP',
       'DESC_FOPRE', 'DESC_OTROS', 'FLE_DOLAR', 'SEG_DOLAR', 'PESO_NETO',
       'PESO_BRUTO', 'UNID_FIQTY', 'UNID_FIDES', 'QUNICOM', 'TUNICOM',
       'SEST_MERCA', 'CANT_BULTO', 'CLASE', 'Razon_Social_Almacen',
       'Jurisdiccion_Almacen', 'Cpais', 'Dpais', 'Cpuerto', 'Dpuerto',
       'Descripcion_Aduana', 'Clase_Bulto', 'Codigo', 'Razon Social'],
      dtype='object')

In [148]:
df_final


KeyboardInterrupt



In [20]:
try:
    for index, row in tqdm(df_importaciones.iterrows(),total=len(df_importaciones), desc="Procesando registros"):
        # Revisa la longitud de las columnas y valores para identificar el problema
        columnas = [
            'CODI_ADUAN', 'ANO_PRESE', 'NUME_CORRE', 'FECH_INGSI', 'TIPO_DOCUM', 'LIBR_TRIBU',
            'DNOMBRE', 'CODI_AGENT', 'FECH_LLEGA', 'VIA_TRANSP', 'EMPR_TRANS', 'CODI_ALMA',
            'CADU_MANIF', 'FECH_MANIF', 'NUME_MANIF', 'FECH_RECEP', 'FECH_CANCE', 'TIPO_CANCE',
            'BANC_CANCE', 'CODI_ENFIN', 'DK', 'PAIS_ORIGE', 'PAIS_ADQUI', 'PUER_EMBAR',
            'NUME_SERIE', 'PART_NANDI', 'DESC_COMER', 'DESC_MATCO', 'DESC_USOAP', 'DESC_FOPRE',
            'DESC_OTROS', 'FOB_DOLPOL', 'FLE_DOLAR', 'SEG_DOLAR', 'PESO_NETO', 'PESO_BRUTO',
            'UNID_FIQTY', 'UNID_FIDES', 'QUNICOM', 'TUNICOM', 'SEST_MERCA', 'ADV_DOLAR',
            'IGV_DOLAR', 'ISC_DOLAR', 'IPM_DOLAR', 'DES_DOLAR', 'IPA_DOLAR', 'SAD_DOLAR',
            'DER_ADUM', 'COMM', 'FMOD', 'CANT_BULTO', 'CLASE', 'TRAT_PREFE', 'TIPO_TRAT',
            'CODI_LIBER', 'IMPR_RELIQ'
        ]
        
        valores = [
            row['CODI_ADUAN'], row['ANO_PRESE'], row['NUME_CORRE'], row['FECH_INGSI'],
            row['TIPO_DOCUM'], row['LIBR_TRIBU'], row['DNOMBRE'], row['CODI_AGENT'],
            row['FECH_LLEGA'], row['VIA_TRANSP'], row['EMPR_TRANS'], row['CODI_ALMA'],
            row['CADU_MANIF'], row['FECH_MANIF'], row['NUME_MANIF'], row['FECH_RECEP'],
            row['FECH_CANCE'], row['TIPO_CANCE'], row['BANC_CANCE'], row['CODI_ENFIN'],
            row['DK'], row['PAIS_ORIGE'], row['PAIS_ADQUI'], row['PUER_EMBAR'],
            row['NUME_SERIE'], row['PART_NANDI'], row['DESC_COMER'], row['DESC_MATCO'],
            row['DESC_USOAP'], row['DESC_FOPRE'], row['DESC_OTROS'], row['FOB_DOLPOL'],
            row['FLE_DOLAR'], row['SEG_DOLAR'], row['PESO_NETO'], row['PESO_BRUTO'],
            row['UNID_FIQTY'], row['UNID_FIDES'], row['QUNICOM'], row['TUNICOM'],
            row['SEST_MERCA'], row['ADV_DOLAR'], row['IGV_DOLAR'], row['ISC_DOLAR'],
            row['IPM_DOLAR'], row['DES_DOLAR'], row['IPA_DOLAR'], row['SAD_DOLAR'],
            row['DER_ADUM'], row['COMM'], row['FMOD'], row['CANT_BULTO'], row['CLASE'],
            row['TRAT_PREFE'], row['TIPO_TRAT'], row['CODI_LIBER'], row['IMPR_RELIQ']
        ]
        
        if len(columnas) != len(valores):
            print(f"Discrepancia: {len(columnas)} columnas, {len(valores)} valores")
            break  

        try:
            cursor.execute(f"""
                INSERT INTO ImportacionesA (
                    {", ".join(columnas)}
                ) VALUES ({", ".join(["?" for _ in columnas])})
            """, valores)
        except Exception as e:
            print(f"Error al insertar el registro en el índice {index}: {e}")
            for col, val in zip(columnas, valores):
                try:
                    cursor.execute(f"SELECT CAST(? AS VARCHAR(255))", (val,))
                except Exception as col_error:
                    print(f"Problema en la columna '{col}' con valor '{val}': {col_error}")
                    break  

    conn.commit()
except ValueError as e:
    print("Error general al procesar los registros:", e)


Procesando registros: 100%|██████████| 7141397/7141397 [2:39:01<00:00, 748.43it/s]   


### Importaciones B

In [5]:
df_mb = pd.read_csv('ImportacionB.csv',encoding='utf-8', sep=';')

In [6]:
df_mb.columns

Index(['CODI_ADUAN', 'ANO_PRESE', 'NUME_CORRE', 'NUME_SECUP', 'NUME_SECUF',
       'FECH_INGSI', 'CODI_PROVE', 'NOMB_PROVE', 'PART_NANDI', 'NUME_ITEM',
       'VFOB_ITEM', 'NOMB_COMER', 'MARC_COMER', 'MODE_MERCD', 'CANT_MERCD',
       'UNID_MERCD', 'CARA_TIPO', 'PAIS_ORIGE', 'ARO_ANO', 'ESTA_MERCD',
       'VFOB_UNITA', 'TERM_TRANS', 'LUGA_TRANS'],
      dtype='object')

In [7]:
print("Campos con nan")
for c in df_mb.columns:
    x=sum(df_mb[c].isna())
    if x!=0:
        print(c, x)

Campos con nan
NOMB_COMER 27498
MARC_COMER 69
MODE_MERCD 1621
CARA_TIPO 8625
PAIS_ORIGE 2
ARO_ANO 1893


In [8]:
# Eliminamos duplicados y reemplazamos nan (ocasionan problemas al momento de subir la base)
df_mb=df_mb.drop_duplicates()
df_mb=df_mb.fillna(0)

In [9]:
df_mb.head()

,CODI_ADUAN,ANO_PRESE,NUME_CORRE,NUME_SECUP,NUME_SECUF,FECH_INGSI,CODI_PROVE,NOMB_PROVE,PART_NANDI,NUME_ITEM,...,MODE_MERCD,CANT_MERCD,UNID_MERCD,CARA_TIPO,PAIS_ORIGE,ARO_ANO,ESTA_MERCD,VFOB_UNITA,TERM_TRANS,LUGA_TRANS
0,19,24,18,1,1,20240904,No Disponi,No Disponible,8430490000,1,...,S/M,1.0,U,"DE PERFORACION PETROLERA ,USADO,DESARMADO,REFE...",IT,0.0,22,6856211.860,FCA,HUAQUILLAS
1,19,24,18,1,2,20240904,No Disponi,No Disponible,8429510000,2,...,980 G,1.0,U,MONTACARGA (CARGADOR FRONTAL) *9CM01332*,US,0.0,20,106425.000,FCA,HUAQUILLAS
2,19,24,18,1,2,20240904,No Disponi,No Disponible,8431490000,3,...,S/M,2.0,U,INCLUYE ACCESORIOS Y PARTES PARA SU NORMAL FUN...,US,0.0,20,4025.135,FCA,HUAQUILLAS
3,19,24,18,1,3,20240904,No Disponi,No Disponible,8430490000,4,...,S/M,1.0,U,ONE NEW POWER CONTROL ROOM-SCR FOR HH220,NL,0.0,20,835058.180,FCA,HUAQUILLAS
4,19,24,18,1,4,20240904,No Disponi,No Disponible,8430490000,5,...,9T1000,1.0,U,MUD PUMPS 9T1000DC-9T1000-155B,IT,0.0,20,451023.260,FCA,HUAQUILLAS


In [14]:
try:
    for index, row in tqdm(df_mb.iterrows(), total=len(df_mb), desc= "Procesando registros"):
        # Revisa la longitud de las columnas y valores para identificar el problema
        columnas = [
           'CODI_ADUAN', 'ANO_PRESE', 'NUME_CORRE', 'NUME_SECUP',
           'FECH_INGSI', 'CODI_PROVE', 'NOMB_PROVE', 'PART_NANDI', 'NUME_ITEM',
           'VFOB_ITEM', 'NOMB_COMER', 'MARC_COMER', 'MODE_MERCD', 'CANT_MERCD',
           'UNID_MERCD', 'CARA_TIPO', 'PAIS_ORIGE', 'ARO_ANO', 'ESTA_MERCD',
           'VFOB_UNITA'
        ]
        
        valores = [
            row['CODI_ADUAN'], row['ANO_PRESE'], row['NUME_CORRE'],
            row['NUME_SECUF'], row['FECH_INGSI'], row['CODI_PROVE'], row['NOMB_PROVE'],
            row['PART_NANDI'], row['NUME_ITEM'], row['VFOB_ITEM'], row['NOMB_COMER'],
            row['MARC_COMER'], row['MODE_MERCD'], row['CANT_MERCD'], row['UNID_MERCD'],
            row['CARA_TIPO'], row['PAIS_ORIGE'], row['ARO_ANO'], row['ESTA_MERCD'],
            row['VFOB_UNITA']
]

        
        # Asegúrate de que las longitudes coincidan
        if len(columnas) != len(valores):
            print(f"Discrepancia: {len(columnas)} columnas, {len(valores)} valores")
            break  

        try:
            # Inserta el registro en la base de datos
            cursor.execute(f"""
                INSERT INTO ImportacionB (
                    {", ".join(columnas)}
                ) VALUES ({", ".join(["?" for _ in columnas])})
            """, valores)
        except Exception as e:
            # Imprime el índice del registro, la columna y el valor problemático
            print(f"Error al insertar el registro en el índice {index}: {e}")
            for col, val in zip(columnas, valores):
                try:
                    # Prueba insertar cada campo individualmente para aislar el valor problemático
                    cursor.execute(f"SELECT CAST(? AS VARCHAR(255))", (val,))
                except Exception as col_error:
                    print(f"Problema en la columna '{col}' con valor '{val}': {col_error}")
                    break  # Sal del loop una vez encontrado el valor problemático

    conn.commit()
except ValueError as e:
    print("Error general al procesar los registros:", e)



Procesando registros:   0%|          | 0/2543947 [00:00<?, ?it/s]

Procesando registros: 100%|██████████| 2543947/2543947 [11:36<00:00, 3650.38it/s] 


### Exportaciones

In [46]:
df_exp = pd.read_csv('Exportacion.csv',encoding='utf-8', sep=';')

C:\Users\gonza\AppData\Local\Temp\ipykernel_16820\3645491519.py:1: DtypeWarning: Columns (18,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_exp = pd.read_csv('Exportacion.csv',encoding='utf-8', sep=';')


In [47]:
df_exp.columns

Index(['CADU', 'FANO', 'NDCL', 'FNUM', 'FEMB', 'FECH_RECEP', 'NDCLREG', 'FREG',
       'FANOREG', 'CAGE', 'TDOC', 'NDOC', 'DNOMBRE', 'CPAIDES', 'CPUEDES',
       'CVIATRA', 'CUNITRA', 'CEMPTRA', 'DMAT', 'NCON', 'CENTFIN', 'CALM',
       'DNOMPRO', 'DDIRPRO', 'DK', 'DK2', 'NSER', 'PART_NANDI', 'DCOM',
       'DMER2', 'DMER3', 'DMER4', 'DMER5', 'CEST', 'VFOBSERDOL', 'VPESNET',
       'VPESBRU', 'QUNIFIS', 'TUNIFIS', 'QUNICOM', 'TUNICOM', 'UBIGEO',
       'DNOMCON', 'DDIRCON'],
      dtype='object')